## Dataset building + XTTS finetuning and inference

#### Running the demo
To start the demo run the first two cells (ignore pip install errors in the first one)

Then click on the link `Running on public URL: ` when the demo is ready.

#### Downloading the results

You can run cell [3] to zip and download default dataset path

You can run cell [4] to zip and download the latest model you trained

### Installing the requirements

In [ ]:
!rm -rf TTS/ # delete repo to be able to reinstall if needed
!git clone --branch xtts_demo -q https://github.com/coqui-ai/TTS.git
!pip install --use-deprecated=legacy-resolver -q -e TTS
!pip install --use-deprecated=legacy-resolver -q -r TTS/TTS/demos/xtts_ft_demo/requirements.txt
!pip install -q typing_extensions==4.8 numpy==1.26.2

### Running the gradio UI

In [ ]:
!python TTS/TTS/demos/xtts_ft_demo/xtts_demo.py --batch_size 2 --num_epochs 6

### Downloading the dataset

In [ ]:
from google.colab import files

!zip -q -r dataset.zip /tmp/xtts_ft/dataset
files.download('dataset.zip')

### Downloading the model

In [ ]:
from google.colab import files
import os
import glob
import torch

def find_latest_best_model(folder_path):
    search_path = os.path.join(folder_path, '**', 'best_model.pth')
    files = glob.glob(search_path, recursive=True)
    latest_file = max(files, key=os.path.getctime, default=None)
    return latest_file

model_path = find_latest_best_model("/tmp/xtts_ft/run/training/")
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
del checkpoint["optimizer"]
for key in list(checkpoint["model"].keys()):
    if "dvae" in key:
        del checkpoint["model"][key]
torch.save(checkpoint, "model.pth")
model_dir = os.path.dirname(model_path)
files.download(os.path.join(model_dir, 'config.json'))
files.download(os.path.join(model_dir, 'vocab.json'))
files.download('model.pth')

### Copy files to your google drive

The two previous cells are a requirement for this step but it can be much faster

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/XTTS_ft_colab
shutil.copy(os.path.join(model_dir, 'config.json'), "/content/drive/MyDrive/XTTS_ft_colab/config.json")
shutil.copy(os.path.join(model_dir, 'vocab.json'), "/content/drive/MyDrive/XTTS_ft_colab/vocab.json'")
shutil.copy('model.pth', "/content/drive/MyDrive/XTTS_ft_colab/model.pth")
shutil.copy('dataset.zip', "/content/drive/MyDrive/XTTS_ft_colab/dataset.zip")